In [2]:
# Import Python Libraries
import numpy as np
import pandas as pd
import requests
from IPython.display import display
from collections import Counter

# Finance data sources
import yfinance as yf
import pandas_datareader as pdr
# import eurostat
# import talib

# Data visualization libraries
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt

import time
from datetime import date
from io import StringIO

# Question 1: IPO Filings Web Scraping and Data Processing
What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context)



In [3]:
import pandas as pd
import requests
from io import StringIO


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_filings = pd.read_html(StringIO(response.text))
ipo_filings = ipo_filings[0]

ipo_filings.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"May 20, 2024",TEM,"Tempus AI, Inc.",-,-
1,"May 17, 2024",TLX,Telix Pharmaceuticals Limited,-,-
2,"May 17, 2024",RAPP,"Rapport Therapeutics, Inc.",-,-
3,"May 14, 2024",ICON,Icon Energy Corp.,$4.00 - $6.00,1250000
4,"May 13, 2024",FCHS,"First Choice Healthcare Solutions, Inc.",$5.00,1200000


In [4]:
# total sum 
# we need avg price of each 
ipo_filings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     333 non-null    object
 1   Symbol          333 non-null    object
 2   Company Name    333 non-null    object
 3   Price Range     333 non-null    object
 4   Shares Offered  333 non-null    object
dtypes: object(5)
memory usage: 13.1+ KB


In [5]:
#changing all column names
ipo_filings.columns = ipo_filings.columns.str.replace(' ','_').str.lower()

# convert to datetime
ipo_filings['filing_date'] = pd.to_datetime(ipo_filings['filing_date'], format='%b %d, %Y')

# convert shares offered to float64
ipo_filings['shares_offered'] = pd.to_numeric(ipo_filings['shares_offered'], errors='coerce')

#fixing price_range column
ipo_filings['price_range'] = ipo_filings['price_range'].str.replace('$',''). replace('-', 'NaN')

def calculate_avg_price(price_range):
    if not price_range:
        return np.nan
    if '-' in price_range:
        min_price, max_price = map(float, price_range.split(' - '))
        return (min_price + max_price) / 2
    else:
        return float(price_range)

#creating average price 
ipo_filings['avg_price'] = ipo_filings['price_range'].apply(calculate_avg_price)

ipo_filings['shares_offered_value'] = ipo_filings['avg_price'] * ipo_filings['shares_offered']


ipo_filings

,filing_date,symbol,company_name,price_range,shares_offered,avg_price,shares_offered_value
0,2024-05-20,TEM,"Tempus AI, Inc.",NaN,NaN,NaN,NaN
1,2024-05-17,TLX,Telix Pharmaceuticals Limited,NaN,NaN,NaN,NaN
2,2024-05-17,RAPP,"Rapport Therapeutics, Inc.",NaN,NaN,NaN,NaN
3,2024-05-14,ICON,Icon Energy Corp.,4.00 - 6.00,1250000.0,5.00,6250000.0
4,2024-05-13,FCHS,"First Choice Healthcare Solutions, Inc.",5.00,1200000.0,5.00,6000000.0
...,...,...,...,...,...,...,...
328,2020-01-21,GOXS,"Goxus, Inc.",8.00 - 10.00,1500000.0,9.00,13500000.0
329,2020-01-21,UTXO,"UTXO Acquisition, Inc.",10.00,5000000.0,10.00,50000000.0
330,2019-12-09,LOHA,Loha Co. Ltd,8.00 - 10.00,2500000.0,9.00,22500000.0
331,2019-10-04,ZGHB,China Eco-Materials Group Co. Limited,4.00,4300000.0,4.00,17200000.0


In [14]:
# total amount for all the filings on Fridays of 2023
ipo_filings_2023_friday = ipo_filings[(ipo_filings['filing_date'].dt.dayofweek == 4) & (ipo_filings['filing_date'].dt.year == 2023)]
ipo_filings_2023_friday.head()


,filing_date,symbol,company_name,price_range,shares_offered,avg_price,shares_offered_value
60,2023-12-29,LEC,Lafayette Energy Corp,3.50 - 4.50,1200000.0,4.0,4800000.0
61,2023-12-29,EPSM,Epsium Enterprise Limited,NaN,NaN,NaN,NaN
63,2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,NaN,NaN,NaN,NaN
64,2023-12-22,LZMH,LZ Technology Holdings Limited,NaN,NaN,NaN,NaN
69,2023-12-15,GIT,Going International Holding Company Limited,NaN,NaN,NaN,NaN


In [15]:
ipo_filings_2023_friday.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 60 to 173
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   filing_date           31 non-null     datetime64[ns]
 1   symbol                31 non-null     object        
 2   company_name          31 non-null     object        
 3   price_range           31 non-null     object        
 4   shares_offered        24 non-null     float64       
 5   avg_price             24 non-null     float64       
 6   shares_offered_value  24 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 1.9+ KB


In [20]:
value = ipo_filings_2023_friday['shares_offered_value'].sum() / 1e6
print(f'Filings during 2023, which happened on Fridays is ${value:.0f}m')

Filings during 2023, which happened on Fridays is $281m


### Question 2:  IPOs "Fixed days hold" strategy

**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/).
Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this [link](https://stockanalysis.com/actions/changes/).
Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day).
Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

* You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
* You can use the `DataFrame.describe()` function to get mean, min, max, 25-50-75% quantiles.

Additional:

* You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
* What's your recommendation: Do you suggest pursuing this strategy for an optimal X?